In [1]:
from datetime import datetime

In [2]:
from noteLib import *

In [3]:
from lilcubeLib.lc_api import Lilcube


lc = Lilcube()

creating Lilcube class


In [4]:
left,bottom,right,top = (30.209661,-2.218817,30.246396,-2.163926)

In [5]:
ul_lat = top
ul_lon = left

lr_lat = bottom
lr_lon = right

aoi_ul_geo = (ul_lat, ul_lon)
aoi_lr_geo = (lr_lat, lr_lon)

#date_range = (datetime(2013,7,6), datetime(2013,7,7))
date_range = (datetime(2013,7,6), datetime(2018,7,7))

es_index = 'datacube'
es_type = 'rwanda'

dsets = lc.search(es_index, es_type, aoi_ul_geo, aoi_lr_geo,
             time = date_range, measurements = ('red', 'green', 'blue','nir') )

(datetime.datetime(2013, 7, 6, 0, 0), datetime.datetime(2018, 7, 7, 0, 0))
2013-07-06


In [6]:
lats = (top,bottom)
lons = (left,right)
m = display_map(lats,lons)

In [7]:
m

![lake rumira](http://1.bp.blogspot.com/-1VYQ3gjzLOg/T_m-HmfYqPI/AAAAAAAAADA/cWGE_sDudcE/s1600/lake+rumira.JPG)

In [8]:
# query practice

In [9]:
%%bash
#! /bin/bash
curl --header 'Content-Type: application/json' -XPOST 'http://elastic:9200/datacube/rwanda/_search' -d '{
"query": {
    "bool": {
      "must": [
              {"range": {"creation_dt": {"gte": "2013-07-06"}}},
              {"range": {"creation_dt": {"lt": "2013-07-07"}}}
              ],
      "filter": [ {
"geo_shape": {
                    "footprint": {
                        "shape": {
                            "type": "Polygon",
                            "coordinates": [
                                [
                                    [-2.219, 30.2097],
                                    [-2.1639, 30.2097],
                                    [-2.1639, 30.2464],
                                    [-2.219, 30.2464],
                                    [-2.219, 30.2097]
                                ]
                            ]
                        }
                    }
                }
             }
        ]
    }
  }
}' |python -m json.tool


{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "mp32JWgBtX-6QUcSke2V",
                "_index": "datacube",
                "_score": 2.0,
                "_source": {
                    "creation_dt": "2013-07-06",
                    "footprint": {
                        "coordinates": [
                            [
                                [
                                    -0.39826,
                                    29.888153
                                ],
                                [
                                    -0.39826,
                                    31.933444
                                ],
                                [
                                    -2.499456,
                                    31.933444
                                ],
                                [
                       

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1996  100  1135  100   861  65747  49875 --:--:-- --:--:-- --:--:-- 66764


In [10]:
from elasticsearch_dsl import connections

connections.create_connection(hosts=['elastic:9200'], timeout=20)

<Elasticsearch([{'host': 'elastic', 'port': 9200}])>

In [11]:

from elasticsearch import Elasticsearch
from dcindexLib.elastic_index import connect_elasticsearch

#client = Elasticsearch()

client = connect_elasticsearch()

# print(help(client.search))

response = client.search(
    index="datacube",
    
    body={"query": {
    "bool": {
      "must": [
              {"range": {"creation_dt": {"gte": "2013-07-06"}}},
              {"range": {"creation_dt": {"lt": "2013-07-07"}}}
              ],
      "filter": [ {
"geo_shape": {
                    "footprint": {
                        "shape": {
                            "type": "Polygon",
                            "coordinates": [
                                [
                                    [-2.219, 30.2097],
                                    [-2.1639, 30.2097],
                                    [-2.1639, 30.2464],
                                    [-2.219, 30.2464],
                                    [-2.219, 30.2097]
                                ]
                            ]
                        }
                    }
                }
             }
        ]
    }
  }}
    
   
)

for hit in response['hits']['hits']:
    print(hit['_score'], hit['_source']['red'])



Yay Connect
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013070601T1-SC20181130015946/LC08_L1TP_172061_20130706_20170503_01_T1_sr_band4.tif
2.0 /mnt/rwanda/LC08/172/062/2013/LC081720622013070601T1-SC20181129211703/LC08_L1TP_172062_20130706_20170503_01_T1_sr_band4.tif


In [12]:
from lilcubeLib.lc_geometry import return_aoi_polygon

geojson_polygon = return_aoi_polygon(aoi_ul_geo, aoi_lr_geo)
geojson_polygon

{"coordinates": [[-2.163926, 30.209661], [-2.163926, 30.246396], [-2.218817, 30.246396], [-2.218817, 30.209661], [-2.163926, 30.209661]], "type": "Polygon"}

In [13]:
from geojson import Polygon

help(Polygon)

Help on class Polygon in module geojson.geometry:

class Polygon(Geometry)
 |  Represents an abstract base class for a WGS84 geometry.
 |  
 |  Method resolution order:
 |      Polygon
 |      Geometry
 |      geojson.base.GeoJSON
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  errors(self)
 |      Return validation errors (if any).
 |      Implement in each subclass.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Geometry:
 |  
 |  __init__(self, coordinates=None, crs=None, validate=False, **extra)
 |      Initialises a Geometry object.
 |      
 |      :param coordinates: Coordinates of the Geometry object.
 |      :type coordinates: tuple or list of tuple
 |      :param crs: CRS
 |      :type crs: CRS object
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from Geometry:
 |  
 |  clean_coordinates(coords) from builtins.type
 |  

In [14]:
from lilcubeLib.lc_elastic import return_geo_query_body

mybody = return_geo_query_body(aoi_ul_geo, aoi_lr_geo, time = date_range)

In [15]:
mybody

{'query': {'bool': {'filter': [{'geo_shape': {'footprint': {'shape': {'coordinates': [[[-2.163926,
           30.209661],
          [-2.163926, 30.246396],
          [-2.218817, 30.246396],
          [-2.218817, 30.209661],
          [-2.163926, 30.209661]]],
        'type': 'Polygon'}}}}],
   'must': [{'range': {'creation_dt': {'gte': '2013-07-06'}}},
    {'range': {'creation_dt': {'lt': '2018-07-07'}}}]}}}

In [16]:
from elasticsearch import Elasticsearch
from dcindexLib.elastic_index import connect_elasticsearch

#client = Elasticsearch()

client = connect_elasticsearch()

# print(help(client.search))

response = client.search(
    index="datacube",size=1400,
    body=mybody)

Yay Connect


In [17]:
cnt = 0
for hit in response['hits']['hits']:
    print(hit['_score'], hit['_source']['red'])
    cnt = cnt + 1
    print("CNT=",cnt)

2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013090801T1-SC20181130061025/LC08_L1TP_172061_20130908_20170502_01_T1_sr_band4.tif
CNT= 1
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013082301T1-SC20181130013822/LC08_L1TP_172061_20130823_20170502_01_T1_sr_band4.tif
CNT= 2
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013072201T1-SC20181130020053/LC08_L1TP_172061_20130722_20180526_01_T1_sr_band4.tif
CNT= 3
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013080701T1-SC20181130023422/LC08_L1TP_172061_20130807_20170503_01_T1_sr_band4.tif
CNT= 4
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013102601T1-SC20181130013945/LC08_L1TP_172061_20131026_20180526_01_T1_sr_band4.tif
CNT= 5
2.0 /mnt/rwanda/LC08/172/061/2013/LC081720612013122901T1-SC20181130013745/LC08_L1TP_172061_20131229_20170427_01_T1_sr_band4.tif
CNT= 6
2.0 /mnt/rwanda/LC08/172/061/2017/LC081720612017041201T1-SC20181130015640/LC08_L1TP_172061_20170412_20170501_01_T1_sr_band4.tif
CNT= 7
2.0 /mnt/rwanda/LC08/172/061/2017/LC081720612017091901T

CNT= 105
2.0 /mnt/rwanda/LC08/173/061/2016/LC081730612016031501T1-SC20181201002607/LC08_L1TP_173061_20160315_20170328_01_T1_sr_band4.tif
CNT= 106
2.0 /mnt/rwanda/LC08/173/061/2016/LC081730612016112601T1-SC20181130224417/LC08_L1TP_173061_20161126_20170317_01_T1_sr_band4.tif
CNT= 107
2.0 /mnt/rwanda/LC08/173/061/2016/LC081730612016012701T1-SC20181130235851/LC08_L1TP_173061_20160127_20170330_01_T1_sr_band4.tif
CNT= 108
2.0 /mnt/rwanda/LC08/173/061/2016/LC081730612016100901T1-SC20181130232841/LC08_L1TP_173061_20161009_20170320_01_T1_sr_band4.tif
CNT= 109
2.0 /mnt/rwanda/LC08/173/061/2016/LC081730612016050201T1-SC20181201002146/LC08_L1TP_173061_20160502_20170325_01_T1_sr_band4.tif
CNT= 110
2.0 /mnt/rwanda/LC08/173/061/2015/LC081730612015080401T1-SC20181130232603/LC08_L1TP_173061_20150804_20170406_01_T1_sr_band4.tif
CNT= 111
2.0 /mnt/rwanda/LC08/173/061/2018/LC081730612018020101T1-SC20181201045546/LC08_L1TP_173061_20180201_20180220_01_T1_sr_band4.tif
CNT= 112
2.0 /mnt/rwanda/LC08/173/061/201

CNT= 218
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017030201T1-SC20181201045529/LC08_L1TP_173061_20170302_20170316_01_T1_sr_band4.tif
CNT= 219
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017012901T1-SC20181130232325/LC08_L1TP_173061_20170129_20170214_01_T1_sr_band4.tif
CNT= 220
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017050501T1-SC20181201005409/LC08_L1TP_173061_20170505_20170515_01_T1_sr_band4.tif
CNT= 221
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017080901T1-SC20181201002527/LC08_L1TP_173061_20170809_20170824_01_T1_sr_band4.tif
CNT= 222
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017102801T1-SC20181130232707/LC08_L1TP_173061_20171028_20171108_01_T1_sr_band4.tif
CNT= 223
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017052101T1-SC20181130233041/LC08_L1TP_173061_20170521_20180526_01_T1_sr_band4.tif
CNT= 224
2.0 /mnt/rwanda/LC08/173/061/2017/LC081730612017111301T1-SC20181201004532/LC08_L1TP_173061_20171113_20171121_01_T1_sr_band4.tif
CNT= 225
2.0 /mnt/rwanda/LC08/173/061/201

In [18]:
help(datetime.date)
t0 = str(datetime.date(date_range[0]))
    
t0

Help on method_descriptor:

date(...)
    Return date object with same year, month and day.



'2013-07-06'

In [19]:
help(Elasticsearch)

Help on class Elasticsearch in module elasticsearch.client:

class Elasticsearch(builtins.object)
 |  Elasticsearch low-level client. Provides a straightforward mapping from
 |  Python to ES REST endpoints.
 |  
 |  The instance has attributes ``cat``, ``cluster``, ``indices``, ``ingest``,
 |  ``nodes``, ``snapshot`` and ``tasks`` that provide access to instances of
 |  :class:`~elasticsearch.client.CatClient`,
 |  :class:`~elasticsearch.client.ClusterClient`,
 |  :class:`~elasticsearch.client.IndicesClient`,
 |  :class:`~elasticsearch.client.IngestClient`,
 |  :class:`~elasticsearch.client.NodesClient`,
 |  :class:`~elasticsearch.client.SnapshotClient` and
 |  :class:`~elasticsearch.client.TasksClient` respectively. This is the
 |  preferred (and only supported) way to get access to those classes and their
 |  methods.
 |  
 |  You can specify your own connection class which should be used by providing
 |  the ``connection_class`` parameter::
 |  
 |      # create connection to localh

In [20]:
help(Elasticsearch.search)

Help on function search in module elasticsearch.client:

search(self, index=None, doc_type=None, body=None, params=None)
    Execute a search query and get back search hits that match the query.
    `<http://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html>`_
    
    :arg index: A comma-separated list of index names to search; use `_all`
        or empty string to perform the operation on all indices
    :arg doc_type: A comma-separated list of document types to search; leave
        empty to perform the operation on all types
    :arg body: The search definition using the Query DSL
    :arg _source: True or false to return the _source field or not, or a
        list of fields to return
    :arg _source_exclude: A list of fields to exclude from the returned
        _source field
    :arg _source_include: A list of fields to extract and return from the
        _source field
    :arg allow_no_indices: Whether to ignore if a wildcard indices
        expression r

In [21]:
#response

len(response['hits']['hits'])

300